In [2]:
import FinanceDataReader as fdr
import pandas as pd
import random
import sklearn

pd.set_option('display.max_columns', None)

### 국내 주식 종목코드와 층화표본데이터 합치기

In [21]:
# 국내 주식종목의 symbol(종목코드), market, name 가져오기
df_krx = fdr.StockListing('KRX').iloc[:,:3]
df_krx.columns = ['거래소코드','시장구분','회사명_krx']
df_krx = df_krx[df_krx['시장구분'].str.contains('KOSPI|KOSDAQ')]

# 상장폐지된 기업도 가져오기(KRX stock delisting symbol list : 상장폐지 종목 전체 리스트)
krx_delisting = fdr.StockListing('KRX-DELISTING').iloc[:,:3]
krx_delisting.columns = ['거래소코드','회사명_krx','시장구분']
krx_delisting = krx_delisting[['거래소코드','시장구분','회사명_krx']]
krx_delisting = krx_delisting[krx_delisting['시장구분'].str.contains('KOSPI|KOSDAQ')]

# 상장기업과 상폐기업 concat하기
krx_all = pd.concat([df_krx, krx_delisting],axis=0)


# 층화표본 할 데이터 불러오기
df = pd.read_csv('./데이터/국면별_데이터셋/feature_select_dataset2/dataset2_확장기ver1_win+robust.csv', encoding='CP949')
# df.drop(columns='Unnamed: 0', inplace=True)

# 거래소코드 6자리 채우기
df['거래소코드'] = df['거래소코드'].astype(str).str.zfill(6)

# 원본데이터프레임에 주식 종목코드 넣기
df_under = pd.merge(df, krx_all, on = '거래소코드', how='left').drop_duplicates()
df_under.drop('회사명_krx',axis=1,inplace=True)

# 기존 컬럼에 시장구분 추가하기
col = df.columns.tolist()
cols = col + ['시장구분']


# 시장구분상 nan으로 뜨는 코넥스 기업이 아주 조금 있어서 삭제함
df_under = df_under.dropna(subset = ['시장구분'])

# 데이터 모양
print(df_under.shape)

(8854, 18)


In [8]:
df.shape

(8866, 17)

In [ ]:
df_under # 코스닥, 코스피 있는 데이터프레임

In [9]:
# 삭제하고 나서도 결측치 있는지 확인
df_under[df_under['시장구분'].isna() == True]

,회사명,거래소코드,회계년도,상장일,상장폐지일_x,부실기업1,부실기업2,부실기업,업종,경기민감도,매출액증가율,정상영업손익증가율,총자본정상영업이익률,자기자본순이익률,CASH FLOW 대 부채비율,CASH FLOW 대 매출액비율,순운전자본회전률,시장구분


### 언더샘플링을 위한 데이터 계층화

In [10]:
# 시장과 경기민감에 따른 계층 비율 찾기
df_under2 = df_under.copy()

# 부실, 정상기업 비율 확인
df_broke = df_under2[df_under2['부실기업']==1]  # 부실기업들 하나씩만 있는게 맞음
df_broke.reset_index(drop=True,inplace=True)

df_norm = df_under2[df_under2['부실기업']==0]   # 정상기업들
df_norm.reset_index(drop=True,inplace=True)

print('부실기업 수  : ', df_broke['회사명'].nunique())
print('정상기업 수 : ', df_norm['회사명'].nunique())

# 계층별 부실기업 비율 확인
df_broke_sens_kospi = df_broke[(df_broke['시장구분']=='KOSPI') & (df_broke['경기민감도']==1)]
df_broke_no_kospi = df_broke[(df_broke['시장구분']=='KOSPI') & (df_broke['경기민감도']==0)]

df_broke_sens_kosdaq = df_broke[(df_broke['시장구분']=='KOSDAQ') & (df_broke['경기민감도'] == 1)]
df_broke_no_kosdaq = df_broke[(df_broke['시장구분']=='KOSDAQ') & (df_broke['경기민감도'] == 0)]

print('\n계층1 - 코스피 민감 부실기업 수 : ', df_broke_sens_kospi['회사명'].nunique())
print('계층2 - 코스피 안민감 부실기업 수 : ', df_broke_no_kospi['회사명'].nunique())
print('계층3 - 코스닥 민감 부실기업 수 : ', df_broke_sens_kosdaq['회사명'].nunique())
print('계층4 - 코스닥 안민감 부실기업 수 : ', df_broke_no_kosdaq['회사명'].nunique())

부실기업 수  :  508
정상기업 수 :  1772

계층1 - 코스피 민감 부실기업 수 :  124
계층2 - 코스피 안민감 부실기업 수 :  31
계층3 - 코스닥 민감 부실기업 수 :  283
계층4 - 코스닥 안민감 부실기업 수 :  70


In [13]:
# 데이터 분포 확인(꼭 필요)
########## 중복기업 제거 데이터(위 print)와 데이터row 수가 같은지 확인############
print('\n계층1 - 코스피 민감 부실기업 행 수 : ', df_broke_sens_kospi.shape)
print('계층2 - 코스피 안민감 부실기업 행 수 : ', df_broke_no_kospi.shape)
print('계층3 - 코스닥 민감 부실기업 행 수 : ', df_broke_sens_kosdaq.shape)
print('계층4 - 코스닥 안민감 부실기업 행 수 : ', df_broke_no_kosdaq.shape)


계층1 - 코스피 민감 부실기업 행 수 :  (124, 18)
계층2 - 코스피 안민감 부실기업 행 수 :  (31, 18)
계층3 - 코스닥 민감 부실기업 행 수 :  (283, 18)
계층4 - 코스닥 안민감 부실기업 행 수 :  (70, 18)


In [14]:
# 권역변수를 만들어서 부실기업 broke_KOSPI, broke_KOSDAQ 데이터프레임 생성
for i in df_broke['시장구분'].unique():
    globals()['broke_{}'.format(i)] = df_broke[df_broke['시장구분']==i]
    globals()['broke_{}'.format(i)].reset_index(drop=True,inplace=True)

# 권역변수를 만들어서 정상기업 norm_KOSPI, norm_KOSDAQ 데이터프레임 생성
for i in df_norm['시장구분'].unique():
    globals()['norm_{}'.format(i)] = df_norm[df_norm['시장구분']==i]
    globals()['norm_{}'.format(i)].reset_index(drop=True,inplace=True)


# 데이터프레임 정의
broke_kospi = broke_KOSPI   # 부실기업 코스피
broke_kosdaq = broke_KOSDAQ # 부실기업 코스닥

norm_kospi = norm_KOSPI     # 정상기업 코스피
norm_kosdaq = norm_KOSDAQ   # 정상기업 코스닥


# 부실기업  코스피 경기민감도
broke_kospi_1 = broke_kospi[broke_kospi['경기민감도']==1].reset_index(drop=True)
broke_kospi_0 = broke_kospi[broke_kospi['경기민감도']==0].reset_index(drop=True)

# 부실기업 코스닥 경기민감도
broke_kosdaq_1 = broke_kosdaq[broke_kosdaq['경기민감도']==1].reset_index(drop=True)
broke_kosdaq_0 = broke_kosdaq[broke_kosdaq['경기민감도']==0].reset_index(drop=True)


# 정상기업  코스피 경기민감도
norm_kospi_1 = norm_kospi[norm_kospi['경기민감도']==1].reset_index(drop=True)
norm_kospi_0 = norm_kospi[norm_kospi['경기민감도']==0].reset_index(drop=True)

# 정상기업 코스닥 경기민감도
norm_kosdaq_1 = norm_kosdaq[norm_kosdaq['경기민감도']==1].reset_index(drop=True)
norm_kosdaq_0 = norm_kosdaq[norm_kosdaq['경기민감도']==0].reset_index(drop=True)

In [ ]:
# 이대일로 트레인 데이터를 뽑아서 부실기업의 두배만 뽑음
# 근데 전체 데이터는 만몇행이고 거기의 30% 하면 트레인 개수보다 테스트 개수가 많음


### 층화표본 데이터셋 만들기 _ 코스피 경기민감 정상기업

In [15]:
# 인덱스 컬럼으로 추가
norm_kospi_1_copy = norm_kospi_1.copy()
norm_kospi_1_copy['index'] = norm_kospi_1_copy.index
print('코스피 민감 정상기업 shape :',norm_kospi_1_copy.shape)

# 데이터셋 섞기
shuff_norm_kospi_1_copy=sklearn.utils.shuffle(norm_kospi_1_copy)
print('코스피 민감 정상기업 섞은거 shape :',shuff_norm_kospi_1_copy.shape)

# 회사명 중복되지 않게 랜덤으로 가져오기
ran_norm_kospi_1_copy = shuff_norm_kospi_1_copy['회사명'].drop_duplicates().sample(n=df_broke_sens_kospi['회사명'].nunique())
ran_norm_kospi_1_copy = pd.DataFrame(ran_norm_kospi_1_copy)

# 랜덤 표본 인덱스 컬럼에 추가하기
ran_norm_kospi_1_copy['index'] = ran_norm_kospi_1_copy.index

# 랜덤 표본기업 인덱스 기준으로 row행 추출
ran_norm_kospi_1_copy2 = ran_norm_kospi_1_copy.copy()
ran_norm_kospi_1_copy2 = pd.merge(ran_norm_kospi_1_copy, norm_kospi_1_copy, how='left', on=['회사명', 'index'])

# 부실기업과 정상기업 데이터 비교
print('\n랜덤 코스피 민감 정상기업 shape :',ran_norm_kospi_1_copy2.shape)   # 칼럼 많은건 인덱스 추가해서
print('랜덤 코스피 민감 부실기업 shape :',broke_kospi_1.shape)

코스피 민감 정상기업 shape : (2782, 19)
코스피 민감 정상기업 섞은거 shape : (2782, 19)

랜덤 코스피 민감 정상기업 shape : (124, 19)
랜덤 코스피 민감 부실기업 shape : (124, 18)


### 층화표본 데이터셋 만들기 _ 코스피 경기 안민감 정상기업

In [16]:
# 인덱스 컬럼으로 추가
norm_kospi_0_copy = norm_kospi_0.copy()
norm_kospi_0_copy['index'] = norm_kospi_0_copy.index
print('코스피 안민감 정상기업 shape :',norm_kospi_0_copy.shape)

# 데이터셋 섞기
shuff_norm_kospi_0_copy=sklearn.utils.shuffle(norm_kospi_0_copy)
print('코스피 안민감 정상기업 섞은거 shape :',shuff_norm_kospi_0_copy.shape)

# 회사명 중복되지 않게 랜덤으로 가져오기
ran_norm_kospi_0_copy = shuff_norm_kospi_0_copy['회사명'].drop_duplicates().sample(df_broke_no_kospi['회사명'].nunique())
ran_norm_kospi_0_copy = pd.DataFrame(ran_norm_kospi_0_copy)

# 랜덤 표본 인덱스 컬럼에 추가하기
ran_norm_kospi_0_copy['index'] = ran_norm_kospi_0_copy.index

# 랜덤 표본기업 인덱스 기준으로 row행 추출
ran_norm_kospi_0_copy2 = ran_norm_kospi_0_copy.copy()
ran_norm_kospi_0_copy2 = pd.merge(ran_norm_kospi_0_copy, norm_kospi_0_copy, how='left', on=['회사명', 'index'])

# 부실기업과 정상기업 데이터 비교
print('\n랜덤 코스피 안민감 정상기업 shape :',ran_norm_kospi_0_copy2.shape)   # 칼럼 많은건 인덱스 추가해서
print('랜덤 코스피 안민감 부실기업 shape :',broke_kospi_0.shape)

코스피 안민감 정상기업 shape : (729, 19)
코스피 안민감 정상기업 섞은거 shape : (729, 19)

랜덤 코스피 안민감 정상기업 shape : (31, 19)
랜덤 코스피 안민감 부실기업 shape : (31, 18)


### 층화표본 데이터셋 만들기 _ 코스닥 경기 민감 정상기업

In [17]:
# 인덱스 컬럼으로 추가
norm_kosdaq_1_copy = norm_kosdaq_1.copy()
norm_kosdaq_1_copy['index'] = norm_kosdaq_1_copy.index
print('코스닥 민감 정상기업 shape :',norm_kosdaq_1_copy.shape)

# 데이터셋 섞기
shuff_norm_kosdaq_1_copy=sklearn.utils.shuffle(norm_kosdaq_1_copy)
print('코스닥 민감 정상기업 섞은거 shape :',shuff_norm_kosdaq_1_copy.shape)

# 회사명 중복되지 않게 랜덤으로 가져오기
ran_norm_kosdaq_1_copy = shuff_norm_kosdaq_1_copy['회사명'].drop_duplicates().sample(n=df_broke_sens_kosdaq['회사명'].nunique())
ran_norm_kosdaq_1_copy = pd.DataFrame(ran_norm_kosdaq_1_copy)

# 랜덤 표본 인덱스 컬럼에 추가하기
ran_norm_kosdaq_1_copy['index'] = ran_norm_kosdaq_1_copy.index

# 랜덤 표본기업 인덱스 기준으로 row행 추출
ran_norm_kosdaq_1_copy2 = ran_norm_kosdaq_1_copy.copy()
ran_norm_kosdaq_1_copy2 = pd.merge(ran_norm_kosdaq_1_copy, norm_kosdaq_1_copy, how='left', on=['회사명', 'index'])

# 부실기업과 정상기업 데이터 비교
print('\n랜덤 코스닥 민감 정상기업 shape :',ran_norm_kosdaq_1_copy2.shape)   # 칼럼 많은건 인덱스 추가해서
print('랜덤 코스닥 민감 부실기업 shape :',broke_kosdaq_1.shape)


코스닥 민감 정상기업 shape : (3847, 19)
코스닥 민감 정상기업 섞은거 shape : (3847, 19)

랜덤 코스닥 민감 정상기업 shape : (283, 19)
랜덤 코스닥 민감 부실기업 shape : (283, 18)


### 층화표본 데이터셋 만들기 _ 코스닥 경기 안민감 정상기업

In [18]:
# 인덱스 컬럼으로 추가
norm_kosdaq_0_copy = norm_kosdaq_0.copy()
norm_kosdaq_0_copy['index'] = norm_kosdaq_0_copy.index
print('코스닥 안민감 정상기업 shape :',norm_kosdaq_0_copy.shape)

# 데이터셋 섞기
shuff_norm_kosdaq_0_copy=sklearn.utils.shuffle(norm_kosdaq_0_copy)
print('코스닥 안민감 정상기업 섞은거 shape :',shuff_norm_kosdaq_0_copy.shape)

# 회사명 중복되지 않게 랜덤으로 가져오기
ran_norm_kosdaq_0_copy = shuff_norm_kosdaq_0_copy['회사명'].drop_duplicates().sample(n=df_broke_no_kosdaq['회사명'].nunique())
ran_norm_kosdaq_0_copy = pd.DataFrame(ran_norm_kosdaq_0_copy)

# 랜덤 표본 인덱스 컬럼에 추가하기
ran_norm_kosdaq_0_copy['index'] = ran_norm_kosdaq_0_copy.index

# 랜덤 표본기업 인덱스 기준으로 row행 추출
ran_norm_kosdaq_0_copy2 = ran_norm_kosdaq_0_copy.copy()
ran_norm_kosdaq_0_copy2 = pd.merge(ran_norm_kosdaq_0_copy, norm_kosdaq_0_copy, how='left', on=['회사명', 'index'])

# 부실기업과 정상기업 데이터 비교
print('\n랜덤 코스닥 안민감 정상기업 shape :',ran_norm_kosdaq_0_copy2.shape)   # 칼럼 많은건 인덱스 추가해서
print('랜덤 코스닥 안민감 부실기업 shape :',broke_kosdaq_0.shape)

코스닥 안민감 정상기업 shape : (988, 19)
코스닥 안민감 정상기업 섞은거 shape : (988, 19)

랜덤 코스닥 안민감 정상기업 shape : (70, 19)
랜덤 코스닥 안민감 부실기업 shape : (70, 18)


In [ ]:
# 코스피 경기 민감 : ran_norm_kospi_1_copy2
# 코스피 경기 안민감 : ran_norm_kospi_0_copy2
# 코스닥 경기 민감 : ran_norm_kosdaq_1_copy2
# 코스닥 경기 안민감 : ran_norm_kosdaq_0_copy2

# 부실기업 : df_broke

# (copy_n 잘 확인하기)

In [19]:
# 정상기업들 칼럼 부실기업이랑 맞추기(index 칼럼 삭제)
ran_norm_kospi_1_copy3 = ran_norm_kospi_1_copy2.drop(columns='index')
ran_norm_kospi_0_copy3 = ran_norm_kospi_0_copy2.drop(columns='index')
ran_norm_kosdaq_1_copy3 = ran_norm_kosdaq_1_copy2.drop(columns='index')
ran_norm_kosdaq_0_copy3 = ran_norm_kosdaq_0_copy2.drop(columns='index')

# 정상기업 데이터 모양
print('코스피 민감 정상기업 : ',ran_norm_kospi_1_copy3.shape)
print('코스피 안민감 정상기업 : ',ran_norm_kospi_0_copy3.shape)
print('코스닥 민감 정상기업 : ',ran_norm_kosdaq_1_copy3.shape)
print('코스딕 인민감 정상기업 : ',ran_norm_kosdaq_0_copy3.shape)

################# 부실기업 row수랑 정상기업 층별 합과 같은지 확인 중요!!!!!!!!! ############################
print('\n부실기업 :',df_broke.shape)
print('정상기업 층별 합 : ', ran_norm_kospi_1_copy3.shape[0]+ran_norm_kospi_0_copy3.shape[0]+ran_norm_kosdaq_1_copy3.shape[0]+ran_norm_kosdaq_0_copy3.shape[0])

# 데이터 프레임 총 row수
print('\n부실+정상기업 최종 row수 : ', df_broke.shape[0]+ran_norm_kospi_1_copy3.shape[0]+ran_norm_kospi_0_copy3.shape[0]+ran_norm_kosdaq_1_copy3.shape[0]+ran_norm_kosdaq_0_copy3.shape[0])

코스피 민감 정상기업 :  (124, 18)
코스피 안민감 정상기업 :  (31, 18)
코스닥 민감 정상기업 :  (283, 18)
코스딕 인민감 정상기업 :  (70, 18)

부실기업 : (508, 18)
정상기업 층별 합 :  508

부실+정상기업 최종 row수 :  1016


### 모든 정상기업 층별 데이터를 부실기업과 합치기

In [53]:
############ 위에 부실+정상기업 최종 row수랑 데이터프래임 'Stratified_ran_norm'의 row수 같은지 확인 ####################
Stratified_ran_norm = pd.concat([df_broke, ran_norm_kospi_1_copy3, ran_norm_kospi_0_copy3, ran_norm_kosdaq_1_copy3, ran_norm_kosdaq_0_copy3])
Stratified_ran_norm

,회사명,거래소코드,회계년도,상장일,상장폐지일_x,부실기업1,부실기업2,부실기업,업종,경기민감도,총자본증가율,자기자본증가율,매출액증가율,종업원수증가율,정상영업손익증가율,총자본정상영업이익률,금융비용부담률,1주당순이익,유동비율,당좌비율,부채비율,CASH FLOW 대 부채비율,CASH FLOW 대 매출액비율,총자본투자효율,부가가치율,종업원1인당부가가치율_계산,자기자본회전률,타인자본회전률,시장구분
0,(주)DB하이텍,000990,201112,197512,NaN,1,1,1,제조업,1,-22.363090,-23.942497,-21.219120,5.38,84.548200,-3.34,5.30,-2238.020276,51.15,33.02,289.22,5.87,9.47,18.17,39.45,84.500974,1.792551,0.59,KOSPI
1,(주)HJ중공업,097230,201212,200708,NaN,1,0,1,건설업,1,-6.417615,-8.106284,-11.834000,-5.42,-50.993758,0.78,5.30,-1102.002310,92.89,83.75,283.85,-4.99,-9.54,8.26,21.33,185.285877,1.423261,0.58,KOSPI
2,(주)SH에너지화학,002360,201912,198512,NaN,0,1,1,제조업,1,0.801364,-6.064012,-23.282396,1.28,-249.053849,-2.55,-0.20,-27.804142,1062.09,887.83,24.66,43.59,7.56,11.10,9.73,82.024684,1.378267,6.79,KOSPI
3,(주)SNT에너지,100840,201712,200802,NaN,0,1,1,제조업,1,-13.726966,1.748375,-27.548501,-10.55,594.671706,5.92,-0.10,646.722361,351.97,347.24,28.42,22.15,8.00,7.56,12.33,116.375128,0.794155,1.99,KOSPI
4,(주)STX,011810,201312,199009,NaN,1,1,1,도매 및 소매업,1,-22.363090,-31.512717,-38.674582,-30.12,-134.116419,-10.89,3.01,-2238.020276,62.06,54.71,8.48,10.30,7.79,-3.59,-5.86,-75.622218,4.913661,0.91,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,아이진(주),185490,201612,201511,NaN,0,0,0,"전문, 과학 및 기술 서비스업",0,-22.363090,-28.896634,82.962585,44.64,-66.246434,-10.89,-2.91,-823.935240,1062.09,940.60,14.65,-19.54,-13.66,-3.59,-5.86,-75.622218,0.348038,0.58,KOSDAQ
93,(주)티비씨,033830,201112,201011,NaN,0,0,0,정보통신업,0,4.365894,1.175680,10.110369,-2.08,-21.862407,4.02,-2.91,491.135800,838.12,837.35,8.73,76.07,16.18,23.76,62.95,167.006312,2.097280,5.65,KOSDAQ
94,(주)디지틀조선일보,033130,201412,199708,NaN,0,0,0,정보통신업,0,4.380944,3.704084,-14.277958,-3.78,-17.097255,5.34,-1.96,96.642091,566.52,564.75,10.25,91.00,16.10,32.39,61.62,90.392751,0.590033,5.96,KOSDAQ
95,(주)에프에스엔,214270,201912,201503,NaN,0,0,0,정보통신업,0,3.880485,54.379849,15.955326,13.46,-20.475189,4.00,3.80,89.720742,109.98,109.75,192.70,4.49,4.00,4.94,6.70,50.120000,2.622784,1.05,KOSDAQ


In [54]:
# 데이터 저장
Stratified_ran_norm.to_csv('./데이터/국면별_데이터셋/층화표본/데이터셋1/층화표본_dataset1_general_win+robust.csv', encoding='CP949', index=False)

In [2]:
import FinanceDataReader as fdr
import pandas as pd
import random
import sklearn

pd.set_option('display.max_columns', None)

# train 샘프링 test 무작위

## 2대1 층화표본 
    정상2 : 부실1

### 국내 주식 종목코드와 층화표본데이터 합치기

In [116]:
# 국내 주식종목의 symbol(종목코드), market, name 가져오기
df_krx = fdr.StockListing('KRX').iloc[:,:3]
df_krx.columns = ['거래소코드','시장구분','회사명_krx']
df_krx = df_krx[df_krx['시장구분'].str.contains('KOSPI|KOSDAQ')]

# 상장폐지된 기업도 가져오기(KRX stock delisting symbol list : 상장폐지 종목 전체 리스트)
krx_delisting = fdr.StockListing('KRX-DELISTING').iloc[:,:3]
krx_delisting.columns = ['거래소코드','회사명_krx','시장구분']
krx_delisting = krx_delisting[['거래소코드','시장구분','회사명_krx']]
krx_delisting = krx_delisting[krx_delisting['시장구분'].str.contains('KOSPI|KOSDAQ')]

# 상장기업과 상폐기업 concat하기
krx_all = pd.concat([df_krx, krx_delisting],axis=0)


# 층화표본 할 데이터 불러오기
df = pd.read_csv('./데이터/국면별_데이터셋/feature_select_dataset5/dataset5_수축기_ver2_win+robust.csv', encoding='CP949')
# df.drop(columns='Unnamed: 0', inplace=True)

# 거래소코드 6자리 채우기
df['거래소코드'] = df['거래소코드'].astype(str).str.zfill(6)

# 원본데이터프레임에 주식 종목코드 넣기
df_under = pd.merge(df, krx_all, on = '거래소코드', how='left').drop_duplicates()
df_under.drop('회사명_krx',axis=1,inplace=True)

# 기존 컬럼에 시장구분 추가하기
col = df.columns.tolist()
cols = col + ['시장구분']


# 시장구분상 nan으로 뜨는 코넥스 기업이 아주 조금 있어서 삭제함
df_under = df_under.dropna(subset = ['시장구분'])

# 데이터 모양
print(df_under.shape)

(6607, 19)


In [117]:
# 삭제하고 나서도 결측치 있는지 확인
df_under[df_under['시장구분'].isna() == True]

,회사명,거래소코드,회계년도,상장일,상장폐지일_x,부실기업1,부실기업2,부실기업,업종,경기민감도,매출액증가율,정상영업손익증가율,자기자본정상영업이익률,금융비용부담률,1주당순이익,CASH FLOW 대 부채비율,CASH FLOW 대 매출액비율,총자본투자효율,시장구분


### 언더샘플링을 위한 데이터 계층화

In [118]:
# 시장과 경기민감에 따른 계층 비율 찾기
df_under2 = df_under.copy()

# 부실, 정상기업 비율 확인
df_broke = df_under2[df_under2['부실기업']==1]
df_broke.reset_index(drop=True,inplace=True)

df_norm = df_under2[df_under2['부실기업']==0]
df_norm.reset_index(drop=True,inplace=True)

print('부실기업 수  : ', df_broke['회사명'].nunique())
print('정상기업 수 : ', df_norm['회사명'].nunique())

# 계층별 부실기업 비율 확인
df_broke_sens_kospi = df_broke[(df_broke['시장구분']=='KOSPI') & (df_broke['경기민감도']==1)]
df_broke_no_kospi = df_broke[(df_broke['시장구분']=='KOSPI') & (df_broke['경기민감도']==0)]

df_broke_sens_kosdaq = df_broke[(df_broke['시장구분']=='KOSDAQ') & (df_broke['경기민감도'] == 1)]
df_broke_no_kosdaq = df_broke[(df_broke['시장구분']=='KOSDAQ') & (df_broke['경기민감도'] == 0)]

print('\n계층1 - 코스피 민감 부실기업 수 : ', df_broke_sens_kospi['회사명'].nunique())
print('계층2 - 코스피 안민감 부실기업 수 : ', df_broke_no_kospi['회사명'].nunique())
print('계층3 - 코스닥 민감 부실기업 수 : ', df_broke_sens_kosdaq['회사명'].nunique())
print('계층4 - 코스닥 안민감 부실기업 수 : ', df_broke_no_kosdaq['회사명'].nunique())

부실기업 수  :  488
정상기업 수 :  1731

계층1 - 코스피 민감 부실기업 수 :  135
계층2 - 코스피 안민감 부실기업 수 :  23
계층3 - 코스닥 민감 부실기업 수 :  274
계층4 - 코스닥 안민감 부실기업 수 :  56


In [119]:
# 데이터 분포 확인(꼭 필요)
########## 중복기업 제거 데이터(위 print)와 데이터row 수가 같은지 확인############
print('\n계층1 - 코스피 민감 부실기업 행 수 : ', df_broke_sens_kospi.shape)
print('계층2 - 코스피 안민감 부실기업 행 수 : ', df_broke_no_kospi.shape)
print('계층3 - 코스닥 민감 부실기업 행 수 : ', df_broke_sens_kosdaq.shape)
print('계층4 - 코스닥 안민감 부실기업 행 수 : ', df_broke_no_kosdaq.shape)


계층1 - 코스피 민감 부실기업 행 수 :  (135, 19)
계층2 - 코스피 안민감 부실기업 행 수 :  (23, 19)
계층3 - 코스닥 민감 부실기업 행 수 :  (274, 19)
계층4 - 코스닥 안민감 부실기업 행 수 :  (56, 19)


In [120]:
# 권역변수를 만들어서 부실기업 broke_KOSPI, broke_KOSDAQ 데이터프레임 생성
for i in df_broke['시장구분'].unique():
    globals()['broke_{}'.format(i)] = df_broke[df_broke['시장구분']==i]
    globals()['broke_{}'.format(i)].reset_index(drop=True,inplace=True)

# 권역변수를 만들어서 정상기업 norm_KOSPI, norm_KOSDAQ 데이터프레임 생성
for i in df_norm['시장구분'].unique():
    globals()['norm_{}'.format(i)] = df_norm[df_norm['시장구분']==i]
    globals()['norm_{}'.format(i)].reset_index(drop=True,inplace=True)


# 데이터프레임 정의
broke_kospi = broke_KOSPI   # 부실기업 코스피
broke_kosdaq = broke_KOSDAQ # 부실기업 코스닥

norm_kospi = norm_KOSPI     # 정상기업 코스피
norm_kosdaq = norm_KOSDAQ   # 정상기업 코스닥


# 부실기업  코스피 경기민감도
broke_kospi_1 = broke_kospi[broke_kospi['경기민감도']==1].reset_index(drop=True)
broke_kospi_0 = broke_kospi[broke_kospi['경기민감도']==0].reset_index(drop=True)

# 부실기업 코스닥 경기민감도
broke_kosdaq_1 = broke_kosdaq[broke_kosdaq['경기민감도']==1].reset_index(drop=True)
broke_kosdaq_0 = broke_kosdaq[broke_kosdaq['경기민감도']==0].reset_index(drop=True)


# 정상기업  코스피 경기민감도
norm_kospi_1 = norm_kospi[norm_kospi['경기민감도']==1].reset_index(drop=True)
norm_kospi_0 = norm_kospi[norm_kospi['경기민감도']==0].reset_index(drop=True)

# 정상기업 코스닥 경기민감도
norm_kosdaq_1 = norm_kosdaq[norm_kosdaq['경기민감도']==1].reset_index(drop=True)
norm_kosdaq_0 = norm_kosdaq[norm_kosdaq['경기민감도']==0].reset_index(drop=True)

In [121]:
print(norm_kospi_1['회사명'].nunique())
print(norm_kospi_0['회사명'].nunique())
print(norm_kosdaq_1['회사명'].nunique())
print(norm_kosdaq_0['회사명'].nunique())

529
136
849
217


### 층화표본 2대1 데이터셋 만들기 _ 코스피 경기민감 정상기업

In [122]:
# 인덱스 컬럼으로 추가
norm_kospi_1_copy = norm_kospi_1.copy()
norm_kospi_1_copy['index'] = norm_kospi_1_copy.index
print('코스피 민감 정상기업 shape :',norm_kospi_1_copy.shape)

# 데이터셋 섞기
shuff_norm_kospi_1_copy=sklearn.utils.shuffle(norm_kospi_1_copy)
print('코스피 민감 정상기업 섞은거 shape :',shuff_norm_kospi_1_copy.shape)

# 회사명 중복되지 않게 랜덤으로 가져오기
ran_norm_kospi_1_copy = shuff_norm_kospi_1_copy['회사명'].drop_duplicates().sample(n=(135*2))
ran_norm_kospi_1_copy = pd.DataFrame(ran_norm_kospi_1_copy)

# 랜덤 표본 인덱스 컬럼에 추가하기
ran_norm_kospi_1_copy['index'] = ran_norm_kospi_1_copy.index

# 랜덤 표본기업 인덱스 기준으로 row행 추출
ran_norm_kospi_1_copy2 = ran_norm_kospi_1_copy.copy()
ran_norm_kospi_1_copy2 = pd.merge(ran_norm_kospi_1_copy, norm_kospi_1_copy, how='left', on=['회사명', 'index'])

# 부실기업과 정상기업 데이터 비교
print('\n랜덤 코스피 민감 정상기업 shape :',ran_norm_kospi_1_copy2.shape)   # 칼럼 많은건 인덱스 추가해서
print('랜덤 코스피 민감 부실기업 shape :',broke_kospi_1.shape)

코스피 민감 정상기업 shape : (2029, 20)
코스피 민감 정상기업 섞은거 shape : (2029, 20)

랜덤 코스피 민감 정상기업 shape : (270, 20)
랜덤 코스피 민감 부실기업 shape : (135, 19)


### 층화표본 2대1 데이터셋 만들기 _ 코스피 경기 안민감 정상기업

In [123]:
# 인덱스 컬럼으로 추가
norm_kospi_0_copy = norm_kospi_0.copy()
norm_kospi_0_copy['index'] = norm_kospi_0_copy.index
print('코스피 안민감 정상기업 shape :',norm_kospi_0_copy.shape)

# 데이터셋 섞기
shuff_norm_kospi_0_copy=sklearn.utils.shuffle(norm_kospi_0_copy)
print('코스피 안민감 정상기업 섞은거 shape :',shuff_norm_kospi_0_copy.shape)

# 회사명 중복되지 않게 랜덤으로 가져오기
ran_norm_kospi_0_copy = shuff_norm_kospi_0_copy['회사명'].drop_duplicates().sample(23*2)
ran_norm_kospi_0_copy = pd.DataFrame(ran_norm_kospi_0_copy)

# 랜덤 표본 인덱스 컬럼에 추가하기
ran_norm_kospi_0_copy['index'] = ran_norm_kospi_0_copy.index

# 랜덤 표본기업 인덱스 기준으로 row행 추출
ran_norm_kospi_0_copy2 = ran_norm_kospi_0_copy.copy()
ran_norm_kospi_0_copy2 = pd.merge(ran_norm_kospi_0_copy, norm_kospi_0_copy, how='left', on=['회사명', 'index'])

# 부실기업과 정상기업 데이터 비교
print('\n랜덤 코스피 안민감 정상기업 shape :',ran_norm_kospi_0_copy2.shape)   # 칼럼 많은건 인덱스 추가해서
print('랜덤 코스피 안민감 정상기업 shape :',broke_kospi_0.shape)

코스피 안민감 정상기업 shape : (538, 20)
코스피 안민감 정상기업 섞은거 shape : (538, 20)

랜덤 코스피 안민감 정상기업 shape : (46, 20)
랜덤 코스피 안민감 정상기업 shape : (23, 19)


### 층화표본 2대1 데이터셋 만들기 _ 코스닥 경기 민감 정상기업

In [124]:
# 인덱스 컬럼으로 추가
norm_kosdaq_1_copy = norm_kosdaq_1.copy()
norm_kosdaq_1_copy['index'] = norm_kosdaq_1_copy.index
print('코스닥 민감 정상기업 shape :',norm_kosdaq_1_copy.shape)

# 데이터셋 섞기
shuff_norm_kosdaq_1_copy=sklearn.utils.shuffle(norm_kosdaq_1_copy)
print('코스닥 민감 정상기업 섞은거 shape :',shuff_norm_kosdaq_1_copy.shape)

# 회사명 중복되지 않게 랜덤으로 가져오기
ran_norm_kosdaq_1_copy = shuff_norm_kosdaq_1_copy['회사명'].drop_duplicates().sample(n=274*2)
ran_norm_kosdaq_1_copy = pd.DataFrame(ran_norm_kosdaq_1_copy)

# 랜덤 표본 인덱스 컬럼에 추가하기
ran_norm_kosdaq_1_copy['index'] = ran_norm_kosdaq_1_copy.index

# 랜덤 표본기업 인덱스 기준으로 row행 추출
ran_norm_kosdaq_1_copy2 = ran_norm_kosdaq_1_copy.copy()
ran_norm_kosdaq_1_copy2 = pd.merge(ran_norm_kosdaq_1_copy, norm_kosdaq_1_copy, how='left', on=['회사명', 'index'])

# 부실기업과 정상기업 데이터 비교
print('\n랜덤 코스닥 민감 정상기업 shape :',ran_norm_kosdaq_1_copy2.shape)   # 칼럼 많은건 인덱스 추가해서
print('랜덤 코스닥 민감 정상기업 shape :',broke_kosdaq_1.shape)


코스닥 민감 정상기업 shape : (2812, 20)
코스닥 민감 정상기업 섞은거 shape : (2812, 20)

랜덤 코스닥 민감 정상기업 shape : (548, 20)
랜덤 코스닥 민감 정상기업 shape : (274, 19)


### 층화표본 2대1 데이터셋 만들기 _ 코스닥 경기 안민감 정상기업

In [125]:
# 인덱스 컬럼으로 추가
norm_kosdaq_0_copy = norm_kosdaq_0.copy()
norm_kosdaq_0_copy['index'] = norm_kosdaq_0_copy.index
print('코스닥 안민감 정상기업 shape :',norm_kosdaq_0_copy.shape)

# 데이터셋 섞기
shuff_norm_kosdaq_0_copy=sklearn.utils.shuffle(norm_kosdaq_0_copy)
print('코스닥 안민감 정상기업 섞은거 shape :',shuff_norm_kosdaq_0_copy.shape)

# 회사명 중복되지 않게 랜덤으로 가져오기
ran_norm_kosdaq_0_copy = shuff_norm_kosdaq_0_copy['회사명'].drop_duplicates().sample(n=56*2)
ran_norm_kosdaq_0_copy = pd.DataFrame(ran_norm_kosdaq_0_copy)

# 랜덤 표본 인덱스 컬럼에 추가하기
ran_norm_kosdaq_0_copy['index'] = ran_norm_kosdaq_0_copy.index

# 랜덤 표본기업 인덱스 기준으로 row행 추출
ran_norm_kosdaq_0_copy2 = ran_norm_kosdaq_0_copy.copy()
ran_norm_kosdaq_0_copy2 = pd.merge(ran_norm_kosdaq_0_copy, norm_kosdaq_0_copy, how='left', on=['회사명', 'index'])

# 부실기업과 정상기업 데이터 비교
print('\n랜덤 코스닥 안민감 정상기업 shape :',ran_norm_kosdaq_0_copy2.shape)   # 칼럼 많은건 인덱스 추가해서
print('랜덤 코스닥 안민감 정상기업 shape :',broke_kosdaq_0.shape)

코스닥 안민감 정상기업 shape : (740, 20)
코스닥 안민감 정상기업 섞은거 shape : (740, 20)

랜덤 코스닥 안민감 정상기업 shape : (112, 20)
랜덤 코스닥 안민감 정상기업 shape : (56, 19)


In [46]:
# 코스피 경기 민감 : ran_norm_kospi_1_copy2
# 코스피 경기 안민감 : ran_norm_kospi_0_copy2
# 코스닥 경기 민감 : ran_norm_kosdaq_1_copy2
# 코스닥 경기 안민감 : ran_norm_kosdaq_0_copy2

# 부실기업 : df_broke

# (copy_n 잘 확인하기)

In [126]:
# 정상기업들 칼럼 부실기업이랑 맞추기(index 칼럼 삭제)
ran_norm_kospi_1_copy3 = ran_norm_kospi_1_copy2.drop(columns='index')
ran_norm_kospi_0_copy3 = ran_norm_kospi_0_copy2.drop(columns='index')
ran_norm_kosdaq_1_copy3 = ran_norm_kosdaq_1_copy2.drop(columns='index')
ran_norm_kosdaq_0_copy3 = ran_norm_kosdaq_0_copy2.drop(columns='index')

# 정상기업 데이터 모양
print('코스피 민감 정상기업 : ',ran_norm_kospi_1_copy3.shape)
print('코스피 안민감 정상기업 : ',ran_norm_kospi_0_copy3.shape)
print('코스닥 민감 정상기업 : ',ran_norm_kosdaq_1_copy3.shape)
print('코스딕 인민감 정상기업 : ',ran_norm_kosdaq_0_copy3.shape)

################# 부실기업 row수랑 정상기업 층별 합과 같은지 확인 중요!!!!!!!!! ############################
print('\n부실기업 :',df_broke.shape)
print('정상기업 층별 합 : ', ran_norm_kospi_1_copy3.shape[0]+ran_norm_kospi_0_copy3.shape[0]+ran_norm_kosdaq_1_copy3.shape[0]+ran_norm_kosdaq_0_copy3.shape[0])

# 데이터 프레임 총 row수
print('\n부실+정상기업 최종 row수 : ', df_broke.shape[0]+ran_norm_kospi_1_copy3.shape[0]+ran_norm_kospi_0_copy3.shape[0]+ran_norm_kosdaq_1_copy3.shape[0]+ran_norm_kosdaq_0_copy3.shape[0])

코스피 민감 정상기업 :  (270, 19)
코스피 안민감 정상기업 :  (46, 19)
코스닥 민감 정상기업 :  (548, 19)
코스딕 인민감 정상기업 :  (112, 19)

부실기업 : (488, 19)
정상기업 층별 합 :  976

부실+정상기업 최종 row수 :  1464


### 모든 정상기업 층별 데이터를 부실기업과 합치기

In [127]:
############ 위에 부실+정상기업 최종 row수랑 데이터프래임 'Stratified_ran_norm'의 row수 같은지 확인 ####################
Stratified_ran_norm = pd.concat([df_broke, ran_norm_kospi_1_copy3, ran_norm_kospi_0_copy3, ran_norm_kosdaq_1_copy3, ran_norm_kosdaq_0_copy3])
Stratified_ran_norm

,회사명,거래소코드,회계년도,상장일,상장폐지일_x,부실기업1,부실기업2,부실기업,업종,경기민감도,매출액증가율,정상영업손익증가율,자기자본정상영업이익률,금융비용부담률,1주당순이익,CASH FLOW 대 부채비율,CASH FLOW 대 매출액비율,총자본투자효율,시장구분
0,(주)DB하이텍,000990,201112,197512,NaN,1,1,1,제조업,1,-1.238855,0.962874,-1.609873,3.961039,-2.009234,-0.230467,0.438090,-0.012348,KOSPI
1,(주)HJ중공업,097230,201212,200708,NaN,1,0,1,건설업,1,-0.805779,-0.527870,-0.402070,3.961039,-1.149571,-0.668414,-1.613164,-0.732159,KOSPI
2,(주)SH에너지화학,002360,201912,198512,NaN,0,1,1,제조업,1,-1.334065,-2.706214,-0.880573,-0.357143,-0.315267,1.290654,0.231994,-0.525876,KOSPI
3,(주)SNT에너지,100840,201712,200802,NaN,0,1,1,제조업,1,-1.530925,7.652417,0.022293,-0.292208,0.208621,0.426051,0.279471,-0.783003,KOSPI
4,(주)강원에너지,114190,201212,200911,NaN,0,1,1,제조업,1,3.694471,0.870986,1.923567,-0.701299,0.256608,-0.705918,-0.902077,0.194661,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,(주)메지온,140410,201212,201201,NaN,0,0,0,"전문, 과학 및 기술 서비스업",0,2.951512,-0.405231,-0.344745,-2.233766,-0.064439,0.353463,0.429458,-1.042310,KOSDAQ
108,(주)솔본,035610,201112,199908,NaN,0,0,0,"전문, 과학 및 기술 서비스업",0,2.807396,7.652417,-0.636146,-0.077922,-0.322053,-0.654300,-0.798489,0.033412,KOSDAQ
109,에스씨아이평가정보(주),036120,201212,199912,NaN,0,0,0,정보통신업,0,-0.716814,1.122595,-0.656847,-0.597403,-0.286753,1.016433,-0.149987,2.282186,KOSDAQ
110,케이지이티에스(주),151860,201712,201307,NaN,0,0,0,"수도, 하수 및 폐기물 처리, 원료 재생업",0,0.390571,0.267117,0.134554,-0.324675,-0.040757,0.945861,1.470731,-0.129290,KOSDAQ


In [128]:
# 데이터셋 구성
print('70%의 값 :', round(1464*0.7))           # train 샘플 뽑을 개수
print('30%의 값 :', 1464-round(1464*0.7))     # test 샘플 뽑을 개수

print('\n전체기업 수 :', df_under2['회사명'].nunique())
print('전체기업-train 기업 수 :', df_under2['회사명'].nunique()-round(1464*0.7))

70%의 값 : 1025
30%의 값 : 439

전체기업 수 : 1936
전체기업-train 기업 수 : 911


### train 셋 나누기

In [129]:
# 인덱스 컬럼으로 추가
Stratified_ran_norm_copy = Stratified_ran_norm.copy()
Stratified_ran_norm_copy['index'] = Stratified_ran_norm_copy.index
print('2:1 정상+기업 shape :',Stratified_ran_norm_copy.shape)

# 데이터셋 섞기
shuff_Stratified_ran_norm_copy=sklearn.utils.shuffle(Stratified_ran_norm_copy)
print('2:1 정상+기업 섞은거 shape :',shuff_Stratified_ran_norm_copy.shape)

# 층화추출 2:1에서 70% 랜덤 샘플링
train_2_1 = shuff_Stratified_ran_norm_copy['회사명'].drop_duplicates().sample(n=round(1464*0.7))
train_2_1 = pd.DataFrame(train_2_1)

# 층화추출 2:1 인덱스 컬럼에 추가하기
train_2_1['index'] = train_2_1.index

# 층화추출 2:1 인덱스 기준으로 row행 추출
train_2_1 = train_2_1.copy()
train_2_1 = pd.merge(train_2_1, Stratified_ran_norm_copy, how='left', on=['회사명', 'index'])

# index컬럼 삭제
train_2_1.drop(columns='index', inplace=True)

# 부실기업과 정상기업 데이터 비교
print('\ntrain 2대1 shape :',train_2_1.shape)   # 칼럼 많은건 인덱스 추가해서
# print('랜덤 코스닥 안민감 정상기업 shape :',broke_kosdaq_0.shape)


# train 2:1 저장
train_2_1.to_csv('./데이터/국면별_데이터셋/층화표본/데이터셋5/층화표본_train_dataset5_수축기ver2_win+robust.csv', encoding='CP949', index=False)

2:1 정상+기업 shape : (1464, 20)
2:1 정상+기업 섞은거 shape : (1464, 20)

train 2대1 shape : (1025, 19)


### test 셋 나누기

In [130]:
# 부실 + 정상 기업 row 데이터
df_under3 = df_under2.copy()

# train에 있는 기업 리스트로 저장
name = train_2_1['회사명'].unique().tolist()

# df_under3의 회사명이 name이 아닌 것 저장
noname = df_under3[~df_under3['회사명'].isin(name)]
print('train회사명 제외 남은 회사 수 :',noname['회사명'].nunique()) # 총 기업 : 1986-1583= 403

# 데이터셋 섞기
shuff_noname=sklearn.utils.shuffle(noname)
print('train회사명 제외 데이터 섞은거 shape :',shuff_noname.shape)

# test 추출 시 train 기업 중복 안되게 추출
test_2_1 = shuff_noname.drop_duplicates(['회사명']).sample(n=439)   
print('train회사명 제외 test 데이터 shape :',test_2_1.shape)


# test 2:1 저장
test_2_1.to_csv('./데이터/국면별_데이터셋/층화표본/데이터셋5/층화표본_test_dataset5_수축기ver2_win+robust.csv', encoding='CP949', index=False)

train회사명 제외 남은 회사 수 : 911
train회사명 제외 데이터 섞은거 shape : (3344, 19)
train회사명 제외 test 데이터 shape : (439, 19)
